In [1]:
## We will get ELECTRA Bert Embedding

In [56]:
!pip install pdfminer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 52.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 67.2 MB/s eta 0:00:00
  Created wheel for pdfminer: filename=pdfminer-20191125-py3-none-any.whl size=6140094 sha256=877d314adaabcde0253e49674638de2fc85c1e4f5a9f91a210df92716c9e78da
  Stored in directory: /root/.cache/pip/wheels/d1/aa/48/370f83a970d62355a2a47d2d640094a64eea932c22edae1891
Successfully built pdfminer


In [5]:
from resume_reader_preprocess import preprocess as cleaner

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
!pip3 install --quiet tensorflow-text



In [7]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text  

# ELCTRA WORD EMBEDDING

https://tfhub.dev/google/collections/electra/1

## We are using Electra Small

In [8]:
# Load the BERT encoder and preprocessing models
preprocess = hub.load('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
bert = hub.load('https://tfhub.dev/google/electra_small/2')

In [9]:
# Define some sentences to feed into the model
sentences = [
  "Here We Go Then, You And I is a 1999 album by Norwegian pop artist Morten Abel. It was Abel's second CD as a solo artist.",
  "The album went straight to number one on the Norwegian album chart, and sold to double platinum.",
  "Ceylon spinach is a common name for several plants and may refer to: Basella alba Talinum fruticosum",
  "A solar eclipse occurs when the Moon passes between Earth and the Sun, thereby totally or partly obscuring the image of the Sun for a viewer on Earth.",
  "A partial solar eclipse occurs in the polar regions of the Earth when the center of the Moon's shadow misses the Earth.",
]

In [10]:
# Convert the sentences to bert inputs
bert_inputs = preprocess(sentences)


In [41]:
def embed(sentences,LIST=True):
  """"
  Sentence should be array []
  LIST: TRUE/FALSE return  format
  """
  
  bert_inputs = preprocess(sentences) # preprocess to make sentence ready for bert_inputs
  bert_outputs =  bert(bert_inputs, training=False) # fitting to the bert
  pooled_output = bert_outputs['pooled_output'] # getting the vectors
  return pooled_output.numpy().tolist()[0] if LIST else pooled_output    # return the python list of vectors


In [66]:
def arr_to_vector(TEXT_ARR,preprocess=True):
  '''
  TEXT_ARR is the array of text
  
  EXAMPLE_FORMAT
  TEXT_ARR = [
    'str1',
    'str2,
    'str3,
    .........
    'strN',
  ]
  '''
  vectors = []
  
  for text in TEXT_ARR:
    ## getting embedding
    embeddings = embed([cleaner(text)])
    vectors.append(embeddings)

  return vectors

In [42]:
vectors = embed(['this is akshat srivastava'])

We will convert the Embedding Into a pandas dataframe which can be feeded to a ml model for training

In [43]:
## loading the data

In [44]:
import pandas as pd
exp = pd.read_excel('EXP_INO.xlsx')
summary = pd.read_excel('SUMMARY_INFO')

In [55]:
## This section has to be mannualy update as per requirements

In [81]:
vectors  = arr_to_vector(exp['TEXT'])

In [82]:
vectors_df = pd.DataFrame(vectors)

In [83]:
vectors_df['text'] = exp['TEXT']

In [84]:
vectors_df['class'] = 'EXP_INFO'
vectors_df['label'] = 1

ADDING SUMMARY

In [85]:
summary.columns

Index(['TEXT'], dtype='object')

In [86]:
summary_vectors = arr_to_vector(summary['TEXT'])


In [87]:
summary_vectors_df = pd.DataFrame(summary_vectors)

In [88]:
summary_vectors_df['text'] = summary['TEXT']

In [89]:
summary_vectors_df['class'] = 'SUMMARY'
summary_vectors_df['label'] = 2

In [90]:
summary_vectors_df

,0,1,2,3,4,5,6,7,8,9,...,249,250,251,252,253,254,255,text,class,label
0,-0.652313,-0.073089,0.373655,-0.684890,0.280955,-0.071669,0.941807,0.887527,0.075080,0.356120,...,-0.598690,-0.709848,-0.406096,-0.933248,0.805515,-0.849985,-0.650614,I’m looking forwarder to working at an IT spec...,SUMMARY,2
1,-0.582935,-0.300918,0.032834,-0.692747,0.493823,-0.315651,0.979037,0.922986,0.164959,-0.211012,...,-0.669116,-0.593123,-0.916411,-0.932305,0.778241,-0.757547,0.438115,"A Certified Tester in ISTQB®, with good Test A...",SUMMARY,2
2,-0.573719,-0.595333,-0.070800,-0.649739,0.300918,-0.046864,0.956760,0.958091,-0.039165,0.100971,...,-0.519970,-0.856285,-0.769358,-0.930716,0.818174,-0.830370,0.218866,I am seeking for a Network Engineer job to use...,SUMMARY,2
3,-0.582935,-0.300918,0.032834,-0.692747,0.493823,-0.315651,0.979037,0.922986,0.164959,-0.211012,...,-0.669116,-0.593123,-0.916411,-0.932305,0.778241,-0.757547,0.438115,"A Certified Tester in ISTQB®, with good Test A...",SUMMARY,2
4,-0.827779,-0.597135,-0.249508,-0.546564,0.456328,-0.506745,0.958283,0.938775,0.242104,-0.407159,...,-0.642221,-0.690733,-0.908750,-0.948755,0.831924,-0.696763,0.508282,Application development armed with proficient ...,SUMMARY,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,-0.684252,-0.248514,-0.203032,-0.410181,0.604967,-0.161729,0.973157,0.926108,-0.046784,-0.276074,...,-0.578024,-0.702118,-0.898691,-0.967335,0.846803,-0.794450,0.541343,echnical Manager with more than 13 Years of Ex...,SUMMARY,2
98,-0.682897,-0.343177,-0.032102,-0.450304,0.451249,-0.269442,0.981591,0.802364,0.167308,-0.459561,...,-0.378251,-0.657226,-0.906327,-0.912552,0.870302,-0.821820,0.156571,Result-oriented and customer service-focused T...,SUMMARY,2
99,-0.767245,-0.665805,-0.372886,-0.452029,0.352288,-0.438177,0.958117,0.871959,-0.030534,-0.648749,...,-0.706166,-0.699756,-0.905572,-0.923705,0.830723,-0.878322,0.377431,"Passionate, and ambitious Scrum Master with a ...",SUMMARY,2
100,-0.879674,-0.569471,0.240774,-0.722428,0.127586,-0.575065,0.872156,0.931049,0.462435,-0.300206,...,-0.443766,-0.788380,-0.877273,-0.986448,0.840114,-0.806206,-0.589011,Determined and knowledgeable software develope...,SUMMARY,2


In [91]:
## Lets merge these two entities

In [92]:
df = pd.concat([summary_vectors_df,vectors_df])

In [93]:
df

,0,1,2,3,4,5,6,7,8,9,...,249,250,251,252,253,254,255,text,class,label
0,-0.652313,-0.073089,0.373655,-0.684890,0.280955,-0.071669,0.941807,0.887527,0.075080,0.356120,...,-0.598690,-0.709848,-0.406096,-0.933248,0.805515,-0.849985,-0.650614,I’m looking forwarder to working at an IT spec...,SUMMARY,2
1,-0.582935,-0.300918,0.032834,-0.692747,0.493823,-0.315651,0.979037,0.922986,0.164959,-0.211012,...,-0.669116,-0.593123,-0.916411,-0.932305,0.778241,-0.757547,0.438115,"A Certified Tester in ISTQB®, with good Test A...",SUMMARY,2
2,-0.573719,-0.595333,-0.070800,-0.649739,0.300918,-0.046864,0.956760,0.958091,-0.039165,0.100971,...,-0.519970,-0.856285,-0.769358,-0.930716,0.818174,-0.830370,0.218866,I am seeking for a Network Engineer job to use...,SUMMARY,2
3,-0.582935,-0.300918,0.032834,-0.692747,0.493823,-0.315651,0.979037,0.922986,0.164959,-0.211012,...,-0.669116,-0.593123,-0.916411,-0.932305,0.778241,-0.757547,0.438115,"A Certified Tester in ISTQB®, with good Test A...",SUMMARY,2
4,-0.827779,-0.597135,-0.249508,-0.546564,0.456328,-0.506745,0.958283,0.938775,0.242104,-0.407159,...,-0.642221,-0.690733,-0.908750,-0.948755,0.831924,-0.696763,0.508282,Application development armed with proficient ...,SUMMARY,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,0.123194,0.242453,-0.637795,-0.154800,0.418953,0.081065,0.969353,0.871856,0.319201,-0.417769,...,-0.370513,-0.120708,0.372422,-0.860972,0.957425,-0.911722,-0.034594,◦ The redesign features to be stateless servic...,EXP_INFO,1
116,-0.809808,-0.393808,0.175685,-0.203900,0.818366,-0.156374,0.973293,0.703052,0.093149,-0.387134,...,-0.159342,-0.378264,-0.842743,-0.963926,0.840388,-0.836424,0.261621,Implement Machine Learning and Deep Learning m...,EXP_INFO,1
117,-0.797691,-0.499897,-0.423831,-0.360940,0.340001,-0.329398,0.961766,0.853355,0.144791,-0.675018,...,-0.597329,-0.778625,-0.784477,-0.927489,0.853566,-0.829298,-0.029520,"Lead the Fundraising and Marketing , from crea...",EXP_INFO,1
118,-0.768767,-0.048708,-0.098362,-0.770770,0.387750,0.093048,0.931601,0.925914,0.022310,-0.273134,...,-0.337366,-0.778744,-0.668170,-0.966712,0.909064,-0.714299,0.183295,Generate a pool of candidates by receiving res...,EXP_INFO,1


In [94]:
df.to_csv('EXP_SUMMARY_WITH_VECTORS.csv',index=False)